# Convolutional Neural Network 모델 정의

In [1]:
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [5]:
layer = nn.Conv2d(in_channels=4, #입력데이타의 channel수
                  out_channels=2, #filter수
                  kernel_size=3, #filter의 크기
                 )

print(layer)

Conv2d(4, 2, kernel_size=(3, 3), stride=(1, 1))


In [4]:
input_data = torch.ones(1,4,3,3) #(데이터개수,channel,height,width)
feature_map = layer(input_data)
print(feature_map.shape)

#[1:데이터개수, 2:out_channels, 1:height,1:width]
#출력하는 것을 통해, 사이즈가 줄어듬을 볼 수 있다.


torch.Size([1, 2, 1, 1])


In [10]:
#파라미터 weight를 조회할 수 있다.
weight=layer.weight  #conv2d의 weight를 조회한다. =>filter를 조회.
weight.shape
#[2:filter의 개수, 4:필터의 채널 개수, 3:필터 height, 3:필터 width]



torch.Size([2, 4, 3, 3])

In [15]:
#파라미터 bias(필터 당 하나씩 붙어주는 값)

bias = layer.bias
print(bias.shape)#2가 나오는데, 이는 필터의 개수와 같다.
bias

torch.Size([2])


Parameter containing:
tensor([0.1525, 0.1640], requires_grad=True)

In [17]:
#첫번째 필터 계산

input_data[0].shape, weight[0].shape

(torch.Size([4, 3, 3]), torch.Size([4, 3, 3]))

In [20]:

#filter를 통해 계산하는 과정. 첫번째 필터를 계산한다.
#여기서 핵심은 채널별로 계산을 한다는 것이다.

ch1=torch.sum(input_data[0,0] * weight[0,0]) #[0:첫번째 필터, 0:첫번째 채널]
ch2=torch.sum(input_data[0,1] * weight[0,1]) #[0:첫번째 필터, 1:두번째 채널]
ch3=torch.sum(input_data[0,2] * weight[0,2]) #[0:첫번째 필터, 2:세번째 채널]
ch4=torch.sum(input_data[0,3] * weight[0,3]) #[0:첫번째 필터, 3:네번째 채널]

#convolution result of the channel을 계산.
result = ch1+ch2+ch3+ch4+bias[0]
result

tensor(0.2992, grad_fn=<AddBackward0>)

In [27]:
p_layer = nn.MaxPool2d(kernel_size=2, #최대값을 추출할 영역 크기
                       #영역 중에서 2*2 영역을 잡아서, 그 중에서의 최대값을 찾아 내는 것이다.
                       stride = 2, #이동 하는 size를 지정해준다. 
                       #보통은 kernel_size와 stride를 같은 값을 지정해서 겹치지 않도록 한다.
                       #이 둘이 겹치면 대표값의 대표성이 사라진다.
                       
                       
                      )


input_data2 = torch.rand(1,4,4)
input_data2

tensor([[[0.4674, 0.0888, 0.8674, 0.9868],
         [0.6524, 0.7556, 0.4409, 0.2806],
         [0.3109, 0.8473, 0.7244, 0.0778],
         [0.2847, 0.2395, 0.5993, 0.3350]]])

In [28]:
result2 = p_layer(input_data2)
result2.shape
#사이즈를 보면 절반으로 줄어들었다. 



#위의 출력값을 4등분해서, 각 등분의 영역에서 가장 큰 값을 추출해서 나타낸 것이다.

torch.Size([1, 2, 2])

# MNIST

In [1]:
#코드를 작성하기 전에, 필수사항들을 import 한다.

import os

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import models, datasets, transforms
from torchinfo import summary

import matplotlib.pyplot as plt
import numpy as np

from module.data import load_mnist_dataset, load_fashion_mnist_dataset
from module.train import fit

device = "cuda" if torch.cuda.is_available() else "cpu"


In [2]:
#hyper parameter

N_EPOCH = 1
BATCH_SIZE = 256
LR = 0.001


## Data 준비

In [3]:
train_loader = load_mnist_dataset("datasets",BATCH_SIZE,True) #저장경로,batch크기,Trainset 여부
test_loader = load_mnist_dataset("datasets",BATCH_SIZE,False)

In [4]:
#데이터를 잘 가져왔나 확인한다
train_loader.dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: datasets
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )

## CNN 모델 정의

In [5]:
#CNN ->Convolution Layer: filter개수(out_channels로 설정) 뒤로 갈수록 크게 잡는다.
#Max Pooling Layer를 이용해서 출력 결과의 size는 줄여나간다.(보통 2배씩 줄인다.)


#conv block을 만드는 패턴
#1. conv + relu + maxpooling
#2. Conv + BatchNorm + ReLU + MaxPooling
##3. Conv + BatchNorm + ReLU + Dropout + MaxPooling


class MNISTCNNModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.b1 = nn.Sequential(
        #Conv2d():3*3 필터,stride와 padding이 1로 같다. 이는 입력 size와 출력 size가 같다는 것을 의미한다.
        nn.Conv2d(1,32,kernel_size=3, stride=1,padding=1),
        nn.BatchNorm2d(32), #channel을 기준을 정규화한다. -> 입력 channel수를 지정한다.
        nn.ReLU(),
        nn.Dropout2d(p=0.3),
        nn.MaxPool2d(kernel_size=2,stride=2)
        #kernel_size와 stride가 같은 겨우에는 stride를 생략할수 있다.
        #MaxPool2d() 에서도 padding을 지정할 수 있다.
        
        )
        self.b2 = nn.Sequential(
        nn.Conv2d(32,64,kernel_size=3,padding="same"), #더 늘려나간다.
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)    
        
        
            
        )
        
        self.b3 = nn.Sequential(
        nn.Conv2d(64,128,kernel_size=3,padding="same"),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Dropout2d(p=0.3),
        nn.MaxPool2d(kernel_size=2,padding=1) #입력: 7*7이고 2등분으로 줄이면 3.5이다.
            #이를 살리기 위해 padding을 지정한다.
        
        )
        
        
        #결과 출력레이터 =>Linear() 사용.
        self.output_block = nn.Sequential(
        #MaxPool2d(), #출력결과를 입력으로 받는다. =>4차원이다.
         #3차원 => 1차원
        nn.Flatten(),
        nn.Linear(in_features=128*4*4,out_features=512),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(512,10) #out=>클래스 개수
        )
        
        
        
        
    def forward(self,X):
        out = self.b1(X)
        out = self.b2(out)
        out = self.b3(out)
        out = self.output_block(out) #이를 통해 결국 10개의 값을 출력받는다.
        #이름이 output_block인 것을 숙지하자.
        return out
    
    

In [6]:
#summary를 통해 간략한 정보를 보도록 하자.

model = MNISTCNNModel()
summary(model,(1,1,28,28))


Layer (type:depth-idx)                   Output Shape              Param #
MNISTCNNModel                            [1, 10]                   --
├─Sequential: 1-1                        [1, 32, 14, 14]           --
│    └─Conv2d: 2-1                       [1, 32, 28, 28]           320
│    └─BatchNorm2d: 2-2                  [1, 32, 28, 28]           64
│    └─ReLU: 2-3                         [1, 32, 28, 28]           --
│    └─Dropout2d: 2-4                    [1, 32, 28, 28]           --
│    └─MaxPool2d: 2-5                    [1, 32, 14, 14]           --
├─Sequential: 1-2                        [1, 64, 7, 7]             --
│    └─Conv2d: 2-6                       [1, 64, 14, 14]           18,496
│    └─BatchNorm2d: 2-7                  [1, 64, 14, 14]           128
│    └─ReLU: 2-8                         [1, 64, 14, 14]           --
│    └─MaxPool2d: 2-9                    [1, 64, 7, 7]             --
├─Sequential: 1-3                        [1, 128, 4, 4]            --
│    └─Co

## Train

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=LR)


result = fit(train_loader,test_loader,model,loss_fn,optimizer,N_EPOCH,
             save_best_model=False,early_stopping=True,device=device,mode="multi")


Epoch[1/1] - Train loss: 0.05175 Train Accucracy: 0.98370 || Validation Loss: 0.04545 Validation Accuracy: 0.98500
78.43084287643433 초
